In [3]:
from text_loader.loader import DataLoader
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import mlflow
mlflow.set_tracking_uri('file:///Users/meissanechami/MLEng-politicalparties-python/data/') #TODO: make as instruction for user env file maybe?
mlflow.set_experiment("MLflow")

2024/03/21 17:47:25 INFO mlflow.tracking.fluent: Experiment with name 'MLflow' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/meissanechami/MLEng-politicalparties-python/data/932178637015296266', creation_time=1711043245962, experiment_id='932178637015296266', last_update_time=1711043245962, lifecycle_stage='active', name='MLflow', tags={}>

### Pre-processing

In [7]:
with mlflow.start_run() as r:
    data_loader = DataLoader(filepath='../data/Tweets.csv')
    processed_features = data_loader.preprocess_tweets()
    labels = data_loader.preprocess_parties() - 1 #TODO: review why is that

    model_name = f"tfidf_vectorizer_latest"
    
    mlflow.sklearn.save_model(data_loader.vectorizer, path="../data/models/vectorizer", exist_ok=True)

    model_name = f"label_encoder_latest"
    
    mlflow.sklearn.save_model(data_loader.encoder, path="../data/models/encoder", exist_ok=True)

    X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.3)

2024/03/21 17:48:40 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/03/21 17:48:42 WARNING mlflow.sklearn: Model was missing function: p

### Train & evaluate

In [9]:
with mlflow.start_run():
        model = XGBClassifier(booster='gbtree', max_depth=6, max_leaves=0)
        model.fit(X_train, y_train)

        train_accuracy = accuracy_score(y_train, model.predict(X_train))
        accuracy = accuracy_score(y_test, model.predict(X_test))

        print("  train_accuracy: %s" % train_accuracy)
        print("  accuracy: %s" % accuracy)

        params = model.get_xgb_params()

        for param in params.keys():
            mlflow.log_param(param, params[param])

        mlflow.log_metric("train_acc", train_accuracy)
        mlflow.log_metric("test_acc", accuracy)

        model_name = f"model_latest"
        mlflow.sklearn.save_model(model, path="../data/models/model", exist_ok=True)

  train_accuracy: 0.7414659132216385
  accuracy: 0.6928563167431281


/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
# TODO: user version tag instead of model_latest